In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import os
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2D, Conv2DTranspose, Flatten, Concatenate, Activation, LeakyReLU
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow_addons.layers import SpectralNormalization, InstanceNormalization
from keras.layers import BatchNormalization
from keras.layers import ZeroPadding2D
from keras.layers import ReLU
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
clr_path = "/content/drive/MyDrive/datas/color"
gry_path = "/content/drive/MyDrive/datas/black"


clr_img_path = []

for img_path in os.listdir(clr_path) :
    clr_img_path.append(os.path.join(clr_path, img_path))

gry_img_path = []

for img_path in os.listdir(gry_path) :
    gry_img_path.append(os.path.join(gry_path, img_path))

clr_img_path.sort()
gry_img_path.sort()


X = []
y = []

for i in tqdm(range(5000), desc="Processing images"):

    img1 = cv2.cvtColor(cv2.imread(clr_img_path[i]), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(gry_img_path[i]), cv2.COLOR_BGR2RGB)

    y.append(img_to_array(Image.fromarray(cv2.resize(img1,(128,128)))))
    X.append(img_to_array(Image.fromarray(cv2.resize(img2,(128,128)))))

clr_img_path = [os.path.join(clr_path, img_name) for img_name in os.listdir(clr_path)]
gry_img_path = [os.path.join(gry_path, img_name) for img_name in os.listdir(gry_path)]

In [ ]:
red_histograms = []
green_histograms = []
blue_histograms = []

for i in range(5000):
    img1 = cv2.cvtColor(cv2.imread(clr_img_path[i]), cv2.COLOR_BGR2RGB)

    red_hist = cv2.calcHist([img1], [0], None, [256], [0, 256])
    green_hist = cv2.calcHist([img1], [1], None, [256], [0, 256])
    blue_hist = cv2.calcHist([img1], [2], None, [256], [0, 256])

    red_hist = red_hist / red_hist.sum()
    green_hist = green_hist / green_hist.sum()
    blue_hist = blue_hist / blue_hist.sum()

    red_histograms.append(red_hist)
    green_histograms.append(green_hist)
    blue_histograms.append(blue_hist)

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.fill_between(range(256), red_histograms[0].flatten(), color='red', alpha=0.5)
plt.title('Red Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.fill_between(range(256), green_histograms[0].flatten(), color='green', alpha=0.5)
plt.title('Green Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.fill_between(range(256), blue_histograms[0].flatten(), color='blue', alpha=0.5)
plt.title('Blue Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


num_images_to_display = 6

for i in range(num_images_to_display):

    clr_img = cv2.cvtColor(cv2.imread(clr_img_path[i]), cv2.COLOR_BGR2RGB)


    red_hist = cv2.calcHist([clr_img], [0], None, [256], [0, 256])
    green_hist = cv2.calcHist([clr_img], [1], None, [256], [0, 256])
    blue_hist = cv2.calcHist([clr_img], [2], None, [256], [0, 256])


    red_hist = red_hist / red_hist.sum()
    green_hist = green_hist / green_hist.sum()
    blue_hist = blue_hist / blue_hist.sum()


    plt.figure(figsize=(15, 6))

    plt.subplot(2, 4, 1)
    plt.imshow(clr_img)
    plt.title('Color Image')
    plt.axis('off')

    plt.subplot(2, 4, 2)
    plt.fill_between(range(256), red_hist.flatten(), color='red', alpha=0.5)
    plt.title('Red Channel Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.subplot(2, 4, 3)
    plt.fill_between(range(256), green_hist.flatten(), color='green', alpha=0.5)
    plt.title('Green Channel Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.subplot(2, 4, 4)
    plt.fill_between(range(256), blue_hist.flatten(), color='blue', alpha=0.5)
    plt.title('Blue Channel Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()


In [ ]:
plt.figure(figsize=(12, 40))

num_samples = 20

for i in range(num_samples):
    x = np.random.randint(0, 3000)

    plt.subplot(num_samples, 2, 2 * i + 1)
    plt.imshow(X[x] / 255.0, cmap='gray')  # Use 'gray' colormap for grayscale images
    plt.axis('off')
    plt.title('Gray Image')

    plt.subplot(num_samples, 2, 2 * i + 2)
    plt.imshow(y[x] / 255.0)
    plt.axis('off')
    plt.title('Color Image')

plt.tight_layout()
plt.show()

In [ ]:
X = np.array(X)
y = np.array(y)

X = (X/127.5) - 1
y = (y/127.5) - 1

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, shuffle = False)
init = RandomNormal(mean = 0.0, stddev = 0.02)

def d_block(x_in, fltr, strd, pad, bn, inorm):
    x = Conv2D(fltr, (4, 4),
               strides=strd,
               padding=pad,
               use_bias=False,
               kernel_initializer=init)(x_in)

    if bn:
        x = BatchNormalization()(x)
    if inorm:
        x = InstanceNormalization()(x)
    x = LeakyReLU(0.2)(x)
    return x

def u_block(x, skip, fltr, strd, pad, bn, inorm):
    x = Conv2DTranspose(fltr, (4, 4),
                        strides=strd,
                        padding=pad,
                        use_bias=False,
                        kernel_initializer=init)(x)

    if bn:
        x = BatchNormalization()(x)
    if inorm:
        x = InstanceNormalization()(x)
    x = ReLU()(x)
    conc_x = Concatenate()([x, skip])

    return conc_x



In [ ]:
def create_patchgan(image_shape):
    input_gen = Input(shape=image_shape)
    input_tar = Input(shape=image_shape)
    combined_inputs = Concatenate()([input_gen, input_tar])

    x64 = d_block(combined_inputs, 64, 2, 'same', False, False)
    x128 = d_block(x64, 128, 2, 'same', False, True)
    x256 = d_block(x128, 256, 2, 'same', True, False)

    padded_x256 = ZeroPadding2D()(x256)
    x512 = d_block(padded_x256, 512, 1, 'valid', True, False)

    padded_x512 = ZeroPadding2D()(x512)
    x1 = Conv2D(1, (4, 4), strides=1, padding='valid', activation='sigmoid', kernel_initializer=init)(padded_x512)

    model = Model(inputs=[input_gen, input_tar], outputs=x1)
    return model

discriminatotr=create_patchgan((128, 128, 3))
discriminatotr.summary()

In [ ]:
def create_mod_unet():
    input_src = Input(shape=(128, 128, 3))

    x64 = d_block(input_src, 64, 2, 'same', False, False)
    x128 = d_block(x64, 128, 2, 'same', True, False)
    x256 = d_block(x128, 256, 2, 'same', True, False)
    x512 = d_block(x256, 512, 2, 'same', True, False)
    d512 = d_block(x512, 512, 2, 'same', True, False)
    e512 = d_block(d512, 512, 2, 'same', True, False)

    f512 = d_block(e512, 512, 2, 'same', True, False)

    u512 = u_block(f512, e512, 512, 2, 'same', True, False)
    u512 = u_block(u512, d512, 512, 2, 'same', True, False)
    u512 = u_block(u512, x512, 512, 2, 'same', True, False)
    u256 = u_block(u512, x256, 256, 2, 'same', True, False)
    u128 = u_block(u256, x128, 128, 2, 'same', True, False)
    u64 = u_block(u128, x64, 64, 2, 'same', False, True)

    generated_image = Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh', kernel_initializer=init)(u64)

    model = Model(inputs=input_src, outputs=generated_image)
    return model


generator = create_mod_unet()
generator.summary()


In [ ]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

train = train.shuffle(buffer_size=400).batch(batch_size=16)
valid = valid.shuffle(buffer_size=400).batch(batch_size=16)

gen_model = create_mod_unet()
dis_models = [create_patchgan((128, 128, 3)), create_patchgan((64, 64, 3)), create_patchgan((32, 32, 3))]

LAMBDA=100
optimizer_params = { "learning_rate": 0.0002,"beta_1": 0.5,"beta_2": 0.999}

gen_opt = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_0 = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_1 = tf.keras.optimizers.Adam(**optimizer_params)
dis_opt_2 = tf.keras.optimizers.Adam(**optimizer_params)

bce_loss = keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def gen_loss(dis_gen_out, target_img, gen_img):
    adv_loss = bce_loss(tf.ones_like(dis_gen_out), dis_gen_out)
    l1_loss = tf.reduce_mean(tf.abs(tf.subtract(target_img, gen_img)))
    total_loss = adv_loss + (LAMBDA * l1_loss)
    return total_loss, adv_loss, l1_loss

def dis_loss(dis_gen_out, dis_target_out):
    gen_loss = bce_loss(tf.zeros_like(dis_gen_out), dis_gen_out)
    target_loss = bce_loss(tf.ones_like(dis_target_out), dis_target_out)
    total_dis_loss = gen_loss + target_loss
    return total_dis_loss

In [ ]:
@tf.function
def train_batch(bw_img, tar_img):
    def resize_and_compute_losses(resized_tar_img, resized_bw_img, resized_gen_img, dis_model, dis_gen_out, tar_img, gen_img):
        dis_out = dis_model([resized_bw_img, resized_tar_img], training=True)
        g_loss, _, _ = gen_loss(dis_gen_out, tar_img, gen_img)
        d_loss = dis_loss(dis_gen_out, dis_out)
        return g_loss, d_loss

    with tf.GradientTape(persistent=True) as g:
        gen_img = gen_model(bw_img, training=True)
        g_losses = []
        d_losses = []

        resolutions = [(128, 128), (64, 64), (32, 32)]

        for resolution, dis_model in zip(resolutions, dis_models):
            resized_tar_img = tf.image.resize(tar_img, resolution)
            resized_bw_img = tf.image.resize(bw_img, resolution)
            resized_gen_img = tf.image.resize(gen_img, resolution)
            dis_gen_out = dis_model([resized_bw_img, resized_gen_img], training=True)
            g_loss, d_loss = resize_and_compute_losses(resized_tar_img, resized_bw_img, resized_gen_img, dis_model, dis_gen_out, tar_img, gen_img)
            g_losses.append(g_loss)
            d_losses.append(d_loss)

        g_total_loss = sum(g_losses)
        d_total_loss = sum(d_losses)

    g_gradients = g.gradient(g_total_loss, gen_model.trainable_variables)
    d_gradients = [g.gradient(d_loss, dis_model.trainable_variables) for d_loss, dis_model in zip(d_losses, dis_models)]

    gen_opt.apply_gradients(zip(g_gradients, gen_model.trainable_variables))
    for dis_opt, d_grad, dis_model in zip([dis_opt_0, dis_opt_1, dis_opt_2], d_gradients, dis_models):
        dis_opt.apply_gradients(zip(d_grad, dis_model.trainable_variables))

In [ ]:
def plot_images(bw_img, gen_img, tar_img):
    plt.figure(figsize=(20, 20))

    images = [bw_img, gen_img, tar_img]
    titles = ['Gray Scale', 'Generated', 'Ground Truth']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.imshow((images[i][0] + 1.0) / 2.0)
        plt.title(titles[i], fontsize=20)
        plt.axis('off')

    plt.show()

for gry, clr in train.take(1) :
    pass

def fit(EPOCHS=400):
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch} out of {EPOCHS}')
        progress_bar = tqdm(train, total=len(train))

        for bw_image, tar_image in progress_bar:
            train_batch(bw_image, tar_image)

        if epoch % 3 == 0:
            gen_image = gen_model(gry, training=True)
            plot_images(gry, gen_image, clr)
fit(EPOCHS = 300)

In [ ]:
def visualize_generated_images(generator, b_w_images, tar_images, num_samples=10):
    generated_images = generator(b_w_images, training=False)

    for i in range(num_samples):
        b_w_image = (b_w_images[i] + 1.0) / 2.0
        generated_image = (generated_images[i] + 1.0) / 2.0
        tar_image = (tar_images[i] + 1.0) / 2.0

        plt.figure(figsize=(20, 20))

        plt.subplot(1, 3, 1)
        plt.imshow(b_w_image)
        plt.title('Gray Scale', fontsize=20)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(generated_image)
        plt.title('Generated ', fontsize=20)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(tar_image)
        plt.title('Grond Truth', fontsize=20)
        plt.axis('off')

        plt.show()


In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr


gen_image = gen_model(gry, training=True)
gen_images_np = ((gen_image.numpy() + 1.0) / 2.0 * 255).astype(np.uint8)
tar_images_np = ((clr.numpy() + 1.0) / 2.0 * 255).astype(np.uint8)

psnr_values = []
ssim_values = []

for gen_img, tar_img in zip(gen_images_np, tar_images_np):
    psnr = compare_psnr(tar_img, gen_img)
    ssim = compare_ssim(tar_img, gen_img, multichannel=True)
    psnr_values.append(psnr)
    ssim_values.append(ssim)

average_psnr = np.mean(psnr_values)
average_ssim = np.mean(ssim_values)

print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")


In [ ]:
def visualize_generated_images_with_metrics(generator, b_w_images, tar_images, num_samples=3):
    generated_images = generator(b_w_images, training=False)

    plt.figure(figsize=(20, 6*num_samples))

    for i in range(num_samples):
        b_w_image = (b_w_images[i] + 1.0) / 2.0
        generated_image = (generated_images[i] + 1.0) / 2.0
        tar_image = (tar_images[i] + 1.0) / 2.0

        gen_img_np = ((generated_image.numpy() + 1.0) / 2.0 * 255).astype(np.uint8)
        tar_img_np = ((tar_image.numpy() + 1.0) / 2.0 * 255).astype(np.uint8)

        psnr = compare_psnr(tar_img_np, gen_img_np)
        ssim = compare_ssim(tar_img_np, gen_img_np, multichannel=True)

        plt.subplot(num_samples, 4, i*4 + 1)
        plt.imshow(b_w_image)
        plt.title('Gray Scale', fontsize=15)
        plt.axis('off')

        plt.subplot(num_samples, 4, i*4 + 2)
        plt.imshow(generated_image)
        plt.title(f'Generated \nPSNR: {psnr:.2f}\nSSIM: {ssim:.4f}', fontsize=15)
        plt.axis('off')

        plt.subplot(num_samples, 4, i*4 + 3)
        plt.imshow(tar_image)
        plt.title('Ground Truth', fontsize=15)
        plt.axis('off')

    plt.tight_layout()
    plt.show()


visualize_generated_images_with_metrics(gen_model, gry, clr)

In [ ]:
model_path = '/content/drive/My Drive/models/chroma.h5'
gen_model.save(model_path)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

generator_model_path = '/content/drive/My Drive/models/chroma.h5'
gen0 = load_model(generator_model_path)

new_image_path = '/content/baby_123.jpg'
new_image = tf.io.read_file(new_image_path)
new_image = tf.image.decode_jpeg(new_image, channels=3)
new_image = tf.image.resize(new_image, (128, 128))
new_image = tf.cast(new_image, dtype=tf.float32) / 127.5 - 1.0
new_image = tf.expand_dims(new_image, axis=0)

colorized_image = gen0(new_image, training=False)
colorized_image = (colorized_image + 1.0) / 2.0

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(new_image[0] * 0.5 + 0.5)
plt.title('Gray Scale')
plt.axis('off')


plt.subplot(1, 3, 2)
plt.imshow(colorized_image[0])
plt.title('Generated Colorized Image')
plt.axis('off')

plt.show()
